In [6]:
import pytest
from urllib.request import urlretrieve

In [7]:
server_url = 'https://atmos.nmsu.edu/PDS/data/'
testfile_url = server_url + 'vcoir2_0001/data/r0009/ir2_20160313_075709_174_l1b_v10.fit'

In [6]:
%%timeit 
urlretrieve(str(testfile_url), 'testfile.fits')

18 s ± 2.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
from tqdm.auto import tqdm

In [22]:
from pathlib import Path
from urlpath import URL
import requests

def url_retrieve(url: str, outfile: Path, chunk_size: int=128):
    R = requests.get(url, stream=True, allow_redirects=True)
    if R.status_code != 200:
        raise ConnectionError(f'could not download {url}\nerror code: {R.status_code}')
    with tqdm.wrapattr(open(outfile, 'wb'), 'write', miniters=1,
                       total=int(R.headers.get('content-length', 0)),
                       desc=str(outfile)) as fd:
        for chunk in R.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [26]:


url_retrieve(URL(testfile_url), Path('testfile.fits'), chunk_size=128)

In [ ]:
eg_link = "https://github.com/tqdm/tqdm/releases/download/v4.46.0/tqdm-4.46.0-py2.py3-none-any.whl"
eg_file = eg_link.split('/')[-1]
response = requests.get(eg_link, stream=True)
with tqdm.wrapattr(open(eg_file, "wb"), "write", miniters=1,
                   total=int(response.headers.get('content-length', 0)),
                   desc=eg_file) as fout:
    for chunk in response.iter_content(chunk_size=4096):
        fout.write(chunk)

In [8]:
%%timeit
url_retrieve(str(testfile_url), Path('testfile.fits'))

18.5 s ± 1.04 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/miniconda3/envs/py37/lib/python3.7/site-packages/requests/api.py
Type:      function


In [11]:
def _download(url, savepath, chunk_size):
    # urlretrieve(str(url), savepath)
    r = requests.get(str(url), stream=True)
    with open(savepath, "wb") as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)


In [14]:
%%timeit

_download(str(testfile_url), 'testfile.fits', 128)

16.1 s ± 1.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit

_download(str(testfile_url), 'testfile.fits', 256)

17.2 s ± 858 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit

_download(str(testfile_url), 'testfile.fits', 512)

16.8 s ± 411 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
